In [7]:
import pickle
import time
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd
import numpy as np
import re
import datetime
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)

### Competitions tables parser

In [3]:
with open('all_matches', 'rb') as f:
    all_matches = pickle.load(f)

In [4]:
df_to_transform = pd.DataFrame(all_matches)

In [5]:
df = ParsingDataPrepare.transform_columns_to_rows(df_to_transform, ['ligue_header', 'comp_id', 'season_id', 'game_utc', 'game_title','goals', 'game_status'])

In [6]:
df.reset_index(inplace=True)
df.rename(columns={'index':'game_id'}, inplace=True)

In [7]:
df

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,game_status
0,1458512,Segunda División,707,306,26.03.2021 20:00,Sporting Gijón - Alcorcón,0:0,Finished
1,1458506,Segunda División,707,306,26.03.2021 22:00,Castellon - Espanyol,1:3,Finished
2,1459331,First League,435,306,26.03.2021 14:00,Mykolaiv - Alians Lypova Dolyna,2:0,Finished
3,1459330,First League,435,306,26.03.2021 14:00,Polessya - Krystal Kherson,1:1,Finished
4,1459334,First League,435,306,26.03.2021 15:00,Kremin - Hirnyk-Sport,1:1,Finished
...,...,...,...,...,...,...,...,...
543580,10934224,Torneo Federal A,455,38,04.11.2013 00:30,Juventud Unida Universitario - Union Mar del Plata,2:0,Finished
543581,10968262,Division di Honor,463,38,04.11.2013 01:00,Nacional da Madeira - Dakota,1:3,Finished
543582,10925170,Primera D Metropolitana,460,38,04.11.2013 19:30,Claypole - Argentino Rosario,0:1,Finished
543583,10960778,Argentino B,458,38,04.11.2013 01:00,Huracan San Rafael - Pacífico,3:0,Finished


In [8]:
top_ligues = ['12', '13', '14', '15', '16', '17', '18', '419', '420', '454', '474', '483', '485', '550', '554', '560', '565', '577', 
              '581', '587', '591','596', '601', '677', '681', '684', '699', '707', '712', '716', '723'] 

In [9]:
top_ligues = non_staddart_tables_comps = ['456', '695', '727']

In [10]:
date_top_ligues = df[df.comp_id.isin(non_staddart_tables_comps)].game_utc
len(date_top_ligues)

5803

In [11]:
def competition_season_ids(df: pd.DataFrame, col_comp='comp_id', col_season='season_id'):
    comp_seasons = df.groupby(col_comp)[col_season].agg(lambda x: x.unique().tolist())
    comp_seasons = comp_seasons.to_dict()
    for key, value in comp_seasons.items():
        comp_seasons[key] = [x for x in value if re.search(r'[0-9]+', str(x))]
        
    return comp_seasons

In [12]:
# comp_seasons = competition_season_ids(df)

In [13]:
top_comp_seasons = dict((x, y) for x, y in comp_seasons.items() if x in top_ligues)

In [14]:
top_comp_seasons

{'456': ['306', '300', '297', '294', '173', '179', '14', '6', '38'],
 '695': ['306', '300', '297', '294', '173', '180', '166', '38'],
 '727': ['298', '295', '177', '181', '179', '180', '166', '38']}

In [15]:
def parsing_competition_tables(comps_dict: dict, start_url='https://soccer365.me/?c=live&a=showtable'):
    all_comps_seasons = list()
    for comp, seasons in comps_dict.items():
        
        all_seasons = list()
        for season in seasons:
            url = start_url + '&competition_id=' + comp + '&season_id=' + season
            html = requests.get(url).content
            soup = BeautifulSoup(html) 
            
            regular_seasons = [x for x in soup.find_all('div', {'class':'live_comptt_bd'})]
            if regular_seasons != []:
                season_index = [i for i, x in enumerate(regular_seasons) if re.findall('regular season', str(x).lower())]
                print(season_index, season, comp)
                soup = soup.find_all('div', {'class':'live_comptt_bd'})[int(season_index[0])]
            
            tb_header = [x.get_text() for x in soup.find_all('th', {'class': 'ctr'})]

            key_pos = ['Pos']
            places = [x.get_text() for x in soup.find_all('div')]
            place_num = [re.findall('[0-9]+', x) for x in places]
            place_num = list(filter(None, place_num))
            places_dict = [dict(zip(key_pos, x)) for x in place_num]

            scores = [x.get_text() for x in soup.find_all('td', {'class': 'ctr'})]  
            each_score = re.split(r'\\n\',', str(scores))
            clear_each_sc = [re.findall(r'[0-9+-]+', x) for x in each_score]
            score_dict = [dict(zip(tb_header, x)) for x in clear_each_sc]
            _ = [x.update(y) for x, y in zip(score_dict, places_dict)]

            teams = [x.get_text() for x in soup.find_all('span')]
            teams_score = dict(zip(teams, score_dict))

            comps_seasons = {'comp_id': comp, 'season_id': season}
            comps_seasons['table'] = teams_score

            all_seasons.append(comps_seasons)
        all_comps_seasons.append(all_seasons)
    
    return all_comps_seasons   

In [16]:
len(top_comp_seasons)

3

In [17]:
# comp_tables = parsing_competition_tables(top_comp_seasons)

In [19]:
# len(comp_tables)

In [26]:
non_standart_tables_comp = {'456': ['306', '297', '294', '173', '179', '14', '38'],'695': ['306', '300', '297', '294', '173', '180']}
# remove comp:seasons = 456:300, 456:6, 695:166, 695:38, '727': ['295', '177', '181', '179', '180', '166', '38']

In [20]:
# comp_tables_with_ns = parsing_competition_tables(non_standart_tables_comp)

In [28]:
len(non_standart_tables_comp)

2

In [49]:
with open('pickle_files/comp_tables', 'wb') as f:
    pickle.dump(comp_tables, f)

In [23]:
# _ = [comp_tables.append(x) for x in comp_tables_with_ns]

In [50]:
with open('pickle_files/comp_tables', 'rb') as f:
    comp_tables = pickle.load(f)

In [51]:
len(comp_tables)

33

In [43]:
score_names = list(list(table.values())[0].keys())
str(score_names)

"['Pld', 'W', 'D', 'L', 'GF', 'GA', '+/-', 'Pts', 'Pos']"

In [42]:
df = pd.DataFrame()
for x in range(0, len(comp_tables)):
    for y in range(0, len(comp_tables[x])):
        comp_id = comp_tables[x][y]['comp_id']
        season_id = comp_tables[x][y]['season_id'] 
        table = comp_tables[x][y]['table']

        all_data = dict()
        for team_name, v_tab in table.items():
            data = pd.DataFrame({
                'team_name': [team_name],
                 'pos': [v_tab['Pos']],
                 'pld': [v_tab['Pld']],
                 'w': [v_tab['W']],
                 'd': [v_tab['D']],
                 'l': [v_tab['L']],
                 'gf': [v_tab['GF']],
                 'ga': [v_tab['GA']],
                 '+/-': [v_tab['+/-']],
                 'pts': [v_tab['Pts']],
                 'comp_id': [comp_id],
                 'season_id': [season_id]
                })
            df = pd.concat([df, data])

df = df.reset_index(drop=True)                    

In [46]:
df

,team_name,pos,pld,w,d,l,gf,ga,+/-,pts,comp_id,season_id
0,Manchester City,1,32,23,5,4,67,23,+44,74,12,306
1,Man Utd,2,31,18,9,4,61,34,+27,63,12,306
2,Leicester City,3,31,17,5,9,55,37,+18,56,12,306
3,West Ham,4,31,16,7,8,51,39,+12,55,12,306
4,Chelsea,5,31,15,9,7,50,31,+19,54,12,306
...,...,...,...,...,...,...,...,...,...,...,...,...
4463,Partick Thistle,8,33,11,8,14,32,39,-7,41,695,180
4464,Inverness CT,9,33,10,10,13,40,43,-3,40,695,180
4465,Hamilton Academical,10,33,9,9,15,36,55,-19,36,695,180
4466,Kilmarnock,11,33,8,8,17,33,54,-21,32,695,180


In [52]:
# with open('pickle_files/df_comp_tables', 'wb') as f:
#     pickle.dump(df, f)

In [53]:
# with open('pickle_files/df_comp_tables', 'rb') as f:
#     df_comp_tables = pickle.load(f)